In [149]:
import numpy as np
import csv
import pandas as pd

import os



In [23]:
# 指定目标文件夹
output_folder = "data_2_origin/LD"
os.makedirs(output_folder, exist_ok=True)

# 加载 npz 文件
data = np.load("simulation/data_LD_6_gn_001.npz")

# 列出文件中的键
print(data.files)  # 显示 npz 文件中的所有键

X = data['X'].flatten()
T = data['T'].flatten()
n = data['n'].flatten()
u = data['u'].flatten()
p = data['p'].flatten()
q = data['q'].flatten()
E = data['E'].flatten()
df = pd.DataFrame({
    'T': T,
    'X': X,
    'n': n,
    'u': u,
    'p': p,
    'q': q,
    'E': E
})
output_path = 'data_2_origin/LD/LD_gn_001.csv'
df.to_csv(output_path, index=False)
# 假设数据有多个键，例如 X, T, n, u, p, q, E
# 将每个键的数据转换为 csv 文件
# for key in data.files:
#     np_data = data[key].flatten()
#     # 检查是否为二维数组
#     if len(np_data.shape) == 2:
#         # 保存为 csv 文件
#         file_path = os.path.join(output_folder, f"LD_gn_3_{key}.csv")
#         pd.DataFrame(np_data).to_csv(file_path, index=False, header=False)
#     else:
#         print(f"{key} is not a 2D array, skipping...")
X.shape

['X', 'T', 'n', 'u', 'p', 'q', 'E']


(400200,)

In [136]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.autograd import grad
from torch.utils.data import DataLoader, TensorDataset
import os
import shutil
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt

In [161]:
# 定义 TransformerWithPhysics 类
class TransformerWithPhysics(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, num_layers, num_heads, dropout=0.1):
        super(TransformerWithPhysics, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads, dropout=dropout)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(1, 0, 2)  # Transformer expects (seq_len, batch_size, hidden_dim)
        x = self.transformer(x)
        x = x.permute(1, 0, 2)  # Convert back to (batch_size, seq_len, hidden_dim)
        x = self.fc_out(x)
        return x

# 初始化模型
input_dim = 2  # 输入维度
hidden_dim = 128  # 隐藏层大小
output_dim = 5  # 输出维度
num_layers = 4  # Transformer层数
num_heads = 8  # 注意力头数

model = TransformerWithPhysics(input_dim, output_dim, hidden_dim, num_layers, num_heads)

# 加载已经训练好的模型参数
snapshot = torch.load("models/ld_tf_ddp_3/gn/0005/snapshot.pt")

model_state_dict = snapshot["model_state_dict"]
if 'module.' in next(iter(model_state_dict.keys())):
    model_state_dict = {k.replace('module.', ''): v for k, v in model_state_dict.items()}
model.load_state_dict(model_state_dict)
# model.load_state_dict(torch.load('models/model_LD_TF/3/transformer_model_epoch_0.pth'))

# 将模型移动到 GPU 上（如果有GPU的话）
device = torch.device('cuda:2' if torch.cuda.is_available() else
                      'cpu')
model.to(device)

# 将模型设置为评估模式，这会关闭 dropout 等层
model.eval()



TransformerWithPhysics(
  (embedding): Linear(in_features=2, out_features=128, bias=True)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in

In [162]:
import os
import random
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

# 释放未使用的显存
torch.cuda.empty_cache()

# 加载数据
data = np.load('simulation/data_LD_6.npz')
X, T = data["X"].flatten(), data["T"].flatten()
Y_true = np.column_stack([(data[key]).flatten() for key in "nupqE"])

# 将 X 和 T 转换为 3D 形状 (batch_size, seq_len, input_dim)
batch_size = 200  # 适当减少批量大小
seq_len = len(X) // batch_size

X = torch.tensor(X, dtype=torch.float32).view(batch_size, seq_len, 1)
T = torch.tensor(T, dtype=torch.float32).view(batch_size, seq_len, 1)

# 使用模型进行预测
Y_pred = []

with torch.no_grad():
    for i in range(batch_size):
        x_tf = torch.cat((T[i:i+1], X[i:i+1]), dim=2).to(device)
        y_pred_batch = model(x_tf).cpu().numpy()
        Y_pred.append(y_pred_batch)

# 将预测结果转换为 NumPy 数组
Y_pred = np.concatenate(Y_pred, axis=0)

# 打印预测结果

Y_pred = Y_pred.reshape(-1, 5)

In [163]:
# Error = abs(Y_true - Y_pred)
data = np.load('simulation/data_LD_6.npz')
X, T = data["X"].flatten(), data["T"].flatten()
# n_ae = Error[:, 0].flatten()
# u_ae = Error[:, 1].flatten()
# p_ae = Error[:, 2].flatten()
# q_ae = Error[:, 3].flatten()
# E_ae = Error[:, 4].flatten()
n_pred = Y_pred[:, 0].flatten()
u_pred = Y_pred[:, 1].flatten()
p_pred = Y_pred[:, 2].flatten()
q_pred = Y_pred[:, 3].flatten()
E_pred = Y_pred[:, 4].flatten()

df = pd.DataFrame({
    'T': T,
    'X': X,
    'n': n,
    'u': u,
    'p': p,
    'q': q,
    'E': E
})
output_path = 'data_2_origin/LD_pred/LD_gn_0005_pred.csv'
df.to_csv(output_path, index=False)

In [159]:
# ind = (T<=10)
# Y_true = Y_true[ind]
# Y_pred = Y_pred[ind]
t, x = T.reshape(2001, 200), X.reshape(2001, 200)
dx = x[0, 1] - x[0, 0]
t_val = t[:, 0]
# ind_t = (t_val<=10)
# t_val = t_val[ind]

t_val

array([0.000e+00, 1.000e-02, 2.000e-02, ..., 1.998e+01, 1.999e+01,
       2.000e+01])

In [108]:
# ind = (t_val<=10)
# ind
# t_val = t_val[ind]

In [164]:
E_true = Y_true[:,-1].reshape(2001, 200)
E_pred = Y_pred[:,-1].reshape(2001, 200)
energy_true = (E_true**2).sum(axis=1)*dx
energy_pred = (E_pred**2).sum(axis=1)*dx

df = pd.DataFrame({
    'T': t_val,
    'Energy_simulation': energy_true,
    'Energy_prediction': energy_pred
})
output_path = 'data_2_origin/LD_pred/LD_gn_0005_rate.csv'

df.to_csv(output_path, index=False)

In [138]:
output_pred_folder = "data_2_origin/LD_gn_3_pred"
os.makedirs(output_pred_folder, exist_ok=True) 

In [139]:
n_rs = Y_pred[:, 0].flatten()
u_rs = Y_pred[:, 1].flatten()
p_rs = Y_pred[:, 2].flatten()
q_rs = Y_pred[:, 3].flatten()
E_rs = Y_pred[:, 4].flatten()
X = data['X'].flatten()
T = data['T'].flatten()
X.shape

(400200,)

In [140]:
df = pd.DataFrame({
    'T': T,
    'X': X,
    'n': n_rs,
    'u': u_rs,
    'p': p_rs,
    'q': q_rs,
    'E': E_rs
})
output_path = 'data_2_origin/LD_gn_3_pred/LD_gn_3_pred.csv'
df.to_csv(output_path, index=False)

In [42]:
data_rs = np.load('simulation/data_LD_6.npz')
X, T = data_rs["X"].flatten(), data_rs["T"].flatten()
Y_full = np.column_stack([(data_rs[key]).flatten() for key in "nupqE"])

#     step = 8
#     ind = np.arange(0, X.shape[0], step)
ind = np.random.randint(0,X.shape,size=4000)
X_train = X[ind]
T_train = T[ind]
Y_train = Y_full[ind]
Y_train.shape

(4000, 5)

In [167]:
T_train = T_train.flatten()
X_train = X_train.flatten()
p_train = Y_train[:, 2].flatten()
q_train = Y_train[:, 3].flatten()
 
df = pd.DataFrame({
    'T': T_train,
    'X': X_train,
    'p': Y_train[:, 2],
    'q': Y_train[:, 3]
})
output_path = 'data_2_origin/LD/LD_rs.csv'

df.to_csv(output_path, index=False)

In [43]:
t, x = T.reshape(2001, 200), X.reshape(2001, 200)
dx = x[0, 1] - x[0, 0]
t_val = t[:, 0]

In [44]:
E_true = Y_true[:,-1].reshape(2001, 200)
E_pred = Y_pred[:,-1].reshape(2001, 200)
energy_true = (E_true**2).sum(axis=1)*dx
energy_pred = (E_pred**2).sum(axis=1)*dx



In [45]:
df = pd.DataFrame({
    'T': t_val,
    'Energy_simulation': energy_true,
    'Energy_prediction': energy_pred
})
output_path = 'data_2_origin/LD_pred/LD_q_rate.csv'

df.to_csv(output_path, index=False)

In [5]:
# 定义 TransformerWithPhysics 类
class TransformerWithPhysics(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, num_layers, num_heads, dropout=0.1):
        super(TransformerWithPhysics, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads, dropout=dropout)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(1, 0, 2)  # Transformer expects (seq_len, batch_size, hidden_dim)
        x = self.transformer(x)
        x = x.permute(1, 0, 2)  # Convert back to (batch_size, seq_len, hidden_dim)
        x = self.fc_out(x)
        return x

# 初始化模型
input_dim = 2  # 输入维度
hidden_dim = 128  # 隐藏层大小
output_dim = 5  # 输出维度
num_layers = 4  # Transformer层数
num_heads = 8  # 注意力头数

model = TransformerWithPhysics(input_dim, output_dim, hidden_dim, num_layers, num_heads)

# 加载已经训练好的模型参数
snapshot = torch.load("models/ld_tf_ddp_3/gn_3/snapshot.pt")

model_state_dict = snapshot["model_state_dict"]
if 'module.' in next(iter(model_state_dict.keys())):
    model_state_dict = {k.replace('module.', ''): v for k, v in model_state_dict.items()}
model.load_state_dict(model_state_dict)
# model.load_state_dict(torch.load('models/model_LD_TF/3/transformer_model_epoch_0.pth'))

# 将模型移动到 GPU 上（如果有GPU的话）
device = torch.device('cuda:2' if torch.cuda.is_available() else
                      'cpu')
model.to(device)

# 将模型设置为评估模式，这会关闭 dropout 等层
model.eval()


TransformerWithPhysics(
  (embedding): Linear(in_features=2, out_features=128, bias=True)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in

In [6]:
import os
import random
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

# 释放未使用的显存
torch.cuda.empty_cache()

# 加载数据
data = np.load('simulation/data_LD_6.npz')
X, T = data["X"].flatten(), data["T"].flatten()
Y_true = np.column_stack([(data[key]).flatten() for key in "nupqE"])

# 将 X 和 T 转换为 3D 形状 (batch_size, seq_len, input_dim)
batch_size = 200  # 适当减少批量大小
seq_len = len(X) // batch_size

X = torch.tensor(X, dtype=torch.float32).view(batch_size, seq_len, 1)
T = torch.tensor(T, dtype=torch.float32).view(batch_size, seq_len, 1)

# 使用模型进行预测
Y_pred = []

with torch.no_grad():
    for i in range(batch_size):
        x_tf = torch.cat((T[i:i+1], X[i:i+1]), dim=2).to(device)
        y_pred_batch = model(x_tf).cpu().numpy()
        Y_pred.append(y_pred_batch)

# 将预测结果转换为 NumPy 数组
Y_pred = np.concatenate(Y_pred, axis=0)

# 打印预测结果

Y_pred = Y_pred.reshape(-1, 5)

In [190]:
def r_squared(y_true, y_pred):
    ss_res = sum((y_true - y_pred) ** 2)  # Residual sum of squares
    ss_tot = sum((y_true - np.mean(y_true)) ** 2)  # Total sum of squares
    return 1 - (ss_res / ss_tot)  # R-squared calculation

R_squared = r_squared(Y_true, Y_pred)

In [191]:
R_sqr = np.average(R_squared)
R_sqr

0.9999233229880389